In [5]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.nn.functional as F

import os
import sys
sys.path.append('.')
sys.path.append('../')
sys.path.append('../../')
import numpy as np
import networkx as nx
from src.utils.distances import (
    distance_frobenius,
    distance_procrustes_LE,
    DeltaConDistance,
    WL_distance
)
import matplotlib.pyplot as plt
import seaborn as sns



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'grakel'

In [ ]:

n_nodes = 100  # number of nodes
n_graphs = 10  # number of graphs to generate for each probability
p_values = np.linspace(0.1, 0.9, 9)  # different probability values
n_experiments = 10  # number of times to repeat each comparison

# Initialize results dictionary
results = {
    'Frobenius': [],
    'Procrustes': [],
    'DeltaCon': [],
    'WL': [],
    'p_values': [],
}

# Run experiments
for p1 in p_values:
    for _ in range(n_experiments):
        # Generate two random graphs with same probability
        G1 = nx.erdos_renyi_graph(n_nodes, p1)
        G2 = nx.erdos_renyi_graph(n_nodes, p1)
        
        # Convert to adjacency matrices
        A1 = nx.to_numpy_array(G1)
        A2 = nx.to_numpy_array(G2)
        
        # Calculate distances
        frob_dist = distance_frobenius(A1, A2)
        proc_dist = distance_procrustes_LE(A1, A2, k=10)
        delta_dist = DeltaConDistance(A1, A2)
        wl_dist = WL_distance(A1, A2, n_iter=5)
        
        # Store results
        results['Frobenius'].append(frob_dist)
        results['Procrustes'].append(proc_dist)
        results['DeltaCon'].append(delta_dist)
        results['WL'].append(wl_dist)
        results['p_values'].append(p1)


In [ ]:
# Plot results
plt.figure(figsize=(12, 8))
for metric in ['Frobenius', 'Procrustes', 'DeltaCon', 'WL']:
    sns.boxplot(x='p_values', y=metric, data={
        'p_values': results['p_values'],
        metric: results[metric]
    })
    plt.title(f'{metric} Distance vs Edge Probability')
    plt.xlabel('Edge Probability (p)')
    plt.ylabel('Distance')
    plt.show()

# Calculate correlations between distances
distance_matrix = np.array([
    results['Frobenius'],
    results['Procrustes'],
    results['DeltaCon'],
    results['WL']
]).T

correlation_matrix = np.corrcoef(distance_matrix.T)
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, 
            annot=True, 
            xticklabels=['Frobenius', 'Procrustes', 'DeltaCon', 'WL'],
            yticklabels=['Frobenius', 'Procrustes', 'DeltaCon', 'WL'])
plt.title('Correlation between Distance Measures')
plt.show()